# Celebrimbor build search

Scope at infantry and arty combat ability plus dev

In [1]:
from collections import Counter, defaultdict
import itertools
from math import comb
from tqdm.notebook import tqdm
import random

In [2]:
%load_ext autoreload
%autoreload 2

from lib.model import *
from lib.build_tools import *
from lib.loader_tools import *

In [3]:
IDEAS = load_ideas([
    'data/gypsy_transformed/ideas.yaml', 
    'data/gypsy_transformed/ideas_flogi.yaml', 
    'data/gypsy_transformed/ideas_flogi_relig.yaml'
])

POLICIES = load_policies([
    'data/gypsy_transformed/policies.yaml'
])

In [4]:
def print_build(build: Build, effects: list[str]):
    print('--------------------------------------------------------------------')
    print(f"Build - score: {build.score} - ideas: {build.ideas}")
    for effect in effects:
        print(f"\t{effect}: {build.total_effect[effect]:.2f}")

def get_ideas_effect(ideas: tuple[str]):
    total_effect = Counter()
    for idea in ideas:
        total_effect.update(IDEAS[idea].effect)
        
    return total_effect

def compute_build(ideas: tuple[str], base_policy_slots=4, debug=False) -> Build:
    ideas_effect = get_ideas_effect(ideas)

    available_policies = get_available_policies(ideas, POLICIES)
    max_policy_slots = get_max_policy_slots(ideas_effect, base_policy_slots)

    adm_max_policy, dip_max_policy, mil_max_policy = get_max_policy_slots(ideas_effect, base_policy_slots)
    micro_management_policies_effect = get_micro_management_policies_effect(ideas, available_policies, max_policy_slots)
    war_policies_effect = get_war_policies_effect(ideas, MILITARY_WEIGHTS, available_policies, max_policy_slots)

    total_effect = Counter()
    total_effect.update(ideas_effect)
    total_effect.update(micro_management_policies_effect)
    total_effect.update(war_policies_effect)


    
    return Build(
        ideas=ideas, 
        score=score(total_effect, [COUNTRY_WEIGHTS, MILITARY_WEIGHTS], debug=debug),
        total_effect=total_effect,
        war_policies_effect=war_policies_effect,
    )

## Build rules

In [5]:
admin_idea_names = [v.name for k, v in IDEAS.items() if v.type == 'ADM']
diplo_idea_names = [v.name for k, v in IDEAS.items() if v.type == 'DIP']
military_idea_names = [v.name for k, v in IDEAS.items() if v.type == 'MIL']

In [6]:
admin_idea_allowed = [
    'innovativeness_ideas',
    'economic_ideas',
    'expansion_ideas',
    'administrative_ideas',
    'humanist_ideas',
    'judiciary',
    'development',
    'strong_men',
    # 'fem_boy', # Commented since the same as strong_men
    # 'public_admin',
    'centralism',
    'decentralism',
    # Uncomment your government type
    'monarchie0',
    # 'republik0',
    # 'aristo0',
    # 'diktatur0',
    # 'horde0',
    # Uncomment your religion
    # 'religious_ideas',
    'katholisch0',
    # 'protestant0',
    # 'reformiert0',
    # 'orthodox0',
    # 'islam0',
    # 'tengri0',
    # 'hindu0',
    # 'confuci0',
    # 'budda0',
    # 'norse0',
    # 'shinto0',
    # 'cathar0',
    # 'coptic0',
    # 'romuva0',
    # 'suomi0',
    # 'jew0',
    # 'slav0',
    # 'helle0',
    # 'mane0',
    # 'animist0',
    # 'feti0',
    # 'zoro0',
    # 'ancli0',
    # 'nahu0',
    # 'mesoam0',
    # 'inti0',
    # 'tote0',
    # 'shia0',
    # 'ibadi0',
    # 'hussite0',
    # 'alche0'
]
admin_not_compatible = [
    ('strong_men', 'fem_boy',),
    ('centralism', 'decentralism',),
]

In [7]:
diplo_idea_allowed = [
    'spy_ideas',
    'dynastic',
    'influence_ideas',
    'trade_ideas',
    'exploration_ideas',
    'maritime_ideas',
    'heavy_ship',
    'galley_ship',
    'trade_ship',
    'colonial_emp',
    'assimilation',
    'sociaty',
    'propaganda',
    'fleet_base',
    'nationalismus',
    'konigreich0',
    'imperialismus'
]
dip_not_compatible = [
    ('heavy_ship', 'galley_ship'),
    ('heavy_ship', 'trade_ship'),
    ('galley_ship', 'trade_ship')
]

In [8]:
military_idea_allowed = [
    'offensive',
    'defensive',
    'quality',
    'quantity',
    'general_staff',
    'standing_army',
    'conscription',
    'merc_army',
    'weapon_quality',
    'fortress',
    'war_production',
    'formation0',
    'militarism',
    'shock_ideas',
    'fire_ideas'
 ]
mil_not_compatible = [
    ('offensive', 'defensive'),
    ('quality', 'quantity'),
    ('standing_army', 'conscription'),
    ('shock_ideas', 'fire_ideas')
]

## Build score

In [9]:
# Country weights are used to determine the score of a country during peace time
COUNTRY_WEIGHTS = {
    'development_cost': 100,               # Each 10% is 10 points
}

# War weights are used to determine the score of a country during war time and to find the best policies
MILITARY_WEIGHTS ={
    'infantry_power': 150,                  # Each 10% is 15 points
    'artillery_power': 150,                 # Each 10% is 15 points
    'discipline': 300,                      # Each 10% is 30 points
}

## Find the best build

In [10]:
builds = defaultdict(dict)

IDEA_COUNT_THRESHOLD = 0.39

def expand_idea_set(idea_set: set):
    idea_set_count = len(idea_set)
    adm_idea_count = len([x for x in idea_set if x in admin_idea_names])
    dip_idea_count = len([x for x in idea_set if x in diplo_idea_names])
    mil_idea_count = len([x for x in idea_set if x in military_idea_names])
    if adm_idea_count / idea_set_count < IDEA_COUNT_THRESHOLD:
        for idea in admin_idea_allowed:
            if idea not in idea_set:
                expanded_idea_set = idea_set.union({idea})
                for rule_A, rule_B in admin_not_compatible:
                    if rule_A in expanded_idea_set and rule_B in expanded_idea_set:
                        break
                else:
                    yield expanded_idea_set
    if dip_idea_count / idea_set_count < IDEA_COUNT_THRESHOLD:
        for idea in diplo_idea_allowed:
            if idea not in idea_set:
                expanded_idea_set = idea_set.union({idea})
                for rule_A, rule_B in dip_not_compatible:
                    if rule_A in expanded_idea_set and rule_B in expanded_idea_set:
                        break
                else:
                    yield expanded_idea_set
    if mil_idea_count / idea_set_count < IDEA_COUNT_THRESHOLD:
        for idea in military_idea_allowed:
            if idea not in idea_set:
                expanded_idea_set = idea_set.union({idea})
                for rule_A, rule_B in mil_not_compatible:
                    if rule_A in expanded_idea_set and rule_B in expanded_idea_set:
                        break
                else:
                    yield expanded_idea_set

## 3-policy builds

In [11]:
total_options = len(admin_idea_allowed) *  len(diplo_idea_allowed) * len(military_idea_allowed)

with tqdm(total=total_options) as pbar:
    for admin_ideas in admin_idea_allowed:
        for diplo_ideas in diplo_idea_allowed:
            for military_ideas in military_idea_allowed:
                idea_list = [admin_ideas,  diplo_ideas,  military_ideas]
                idea_list.sort()
                ideas = tuple(idea_list)
                build = compute_build(ideas)
                builds[3][ideas] = (build)
                pbar.update(1)

  0%|          | 0/3060 [00:00<?, ?it/s]

## 4-policy builds

In [14]:
for ideas in tqdm(builds[3].keys(), total=len(builds[3])):
    for new_idea_set in expand_idea_set(set(ideas)):
        idea_list = list(new_idea_set)
        idea_list.sort()
        new_ideas = tuple(idea_list)
        new_build = compute_build(new_ideas)
        builds[4][new_ideas] = new_build

  0%|          | 0/3060 [00:00<?, ?it/s]

## 5-policy builds

In [15]:
for ideas in tqdm(builds[4].keys(), total=len(builds[4])):
    for new_idea_set in expand_idea_set(set(ideas)):
        idea_list = list(new_idea_set)
        idea_list.sort()
        new_ideas = tuple(idea_list)
        new_build = compute_build(new_ideas)
        builds[5][new_ideas] = new_build

  0%|          | 0/61119 [00:00<?, ?it/s]

## 6-policy builds

In [16]:
total_options = comb(len(admin_idea_allowed), 2) * comb(len(diplo_idea_allowed), 2) * comb(len(military_idea_allowed), 2)

with tqdm(total=total_options) as pbar:
    for admin_ideas in itertools.combinations(admin_idea_allowed, 2):
        for diplo_ideas in itertools.combinations(diplo_idea_allowed, 2):
            for military_ideas in itertools.combinations(military_idea_allowed, 2):
                idea_set = set(admin_ideas + diplo_ideas + military_ideas)
                for rule_A, rule_B in admin_not_compatible + dip_not_compatible + mil_not_compatible:
                    if rule_A in idea_set and rule_B in idea_set:
                        break
                else:
                    idea_list = list(idea_set)
                    idea_list.sort()
                    ideas = tuple(idea_list)
                    build = compute_build(ideas)
                    builds[6][ideas] = build
                pbar.update(1)


  0%|          | 0/942480 [00:00<?, ?it/s]

## Expand on the best builds

In [17]:
EXPAND_BEST_N = 1000
EXPAND_RANDOM_N = 3000

def get_ideas_to_expand(build_list: list[Build], best_n=EXPAND_BEST_N, random_n=EXPAND_RANDOM_N):
    build_list.sort(key=lambda x: x.score, reverse=True)
    best_builds = build_list[:best_n]
    random_builds = random.sample(build_list[best_n:], random_n)
    return best_builds + random_builds


In [18]:
for i in range(6, 11):
    builds_to_expand = get_ideas_to_expand(list(builds[i].values()))
    for build in tqdm(builds_to_expand, desc=f'Expanding builds {i}'):
        for new_idea_set in expand_idea_set(set(build.ideas)):
            idea_list = list(new_idea_set)
            idea_list.sort()
            new_ideas = tuple(idea_list)
            new_build = compute_build(new_ideas)
            builds[i+1][new_ideas] = new_build

Expanding builds 6:   0%|          | 0/4000 [00:00<?, ?it/s]

Expanding builds 7:   0%|          | 0/4000 [00:00<?, ?it/s]

Expanding builds 8:   0%|          | 0/4000 [00:00<?, ?it/s]

Expanding builds 9:   0%|          | 0/4000 [00:00<?, ?it/s]

Expanding builds 10:   0%|          | 0/4000 [00:00<?, ?it/s]

## The best

In [20]:
build6 = list(builds[6].values())
build6.sort(key=lambda x: x.score, reverse=True)
for build in build6[:5]:
    print_build(build, ['development_cost', 'infantry_power', 'artillery_power', 'discipline'])

--------------------------------------------------------------------
Build - score: 217.5 - ideas: ('economic_ideas', 'imperialismus', 'quality', 'sociaty', 'strong_men', 'weapon_quality')
	development_cost: 0.30
	infantry_power: 0.30
	artillery_power: 0.40
	discipline: 0.28
--------------------------------------------------------------------
Build - score: 212.5 - ideas: ('development', 'economic_ideas', 'imperialismus', 'sociaty', 'standing_army', 'weapon_quality')
	development_cost: 0.40
	infantry_power: 0.30
	artillery_power: 0.30
	discipline: 0.28
--------------------------------------------------------------------
Build - score: 210.0 - ideas: ('economic_ideas', 'konigreich0', 'quality', 'sociaty', 'strong_men', 'weapon_quality')
	development_cost: 0.30
	infantry_power: 0.30
	artillery_power: 0.40
	discipline: 0.25
--------------------------------------------------------------------
Build - score: 207.5 - ideas: ('development', 'economic_ideas', 'imperialismus', 'propaganda', 'st

In [19]:
build8 = list(builds[8].values())
build8.sort(key=lambda x: x.score, reverse=True)
for build in build8[:5]:
    print_build(build, ['development_cost', 'infantry_power', 'artillery_power', 'discipline'])

--------------------------------------------------------------------
Build - score: 292.5 - ideas: ('development', 'economic_ideas', 'imperialismus', 'propaganda', 'quality', 'standing_army', 'strong_men', 'weapon_quality')
	development_cost: 0.60
	infantry_power: 0.40
	artillery_power: 0.40
	discipline: 0.38
--------------------------------------------------------------------
Build - score: 287.5 - ideas: ('decentralism', 'development', 'economic_ideas', 'imperialismus', 'propaganda', 'quality', 'standing_army', 'weapon_quality')
	development_cost: 0.70
	infantry_power: 0.40
	artillery_power: 0.40
	discipline: 0.33
--------------------------------------------------------------------
Build - score: 287.5 - ideas: ('development', 'economic_ideas', 'imperialismus', 'konigreich0', 'quality', 'standing_army', 'strong_men', 'weapon_quality')
	development_cost: 0.70
	infantry_power: 0.40
	artillery_power: 0.40
	discipline: 0.33
----------------------------------------------------------------

[]